<br>
<p style="font-size: 45px;
          text-align:center;
          font-family: 'Times New Roman';
          font-variant: small-caps;
          font-weight: bold;
          line-height: 50px;"> Detección de Suciedad en Paneles Solares</p>
          
<p style="font-size: 20px;
text-align:center;
font-family: 'Times New Roman';
font-variant: small-caps;
font-weight: italic;
line-height: 50px;">Parte 1B: Propuesta Solución 2 Segmentación </p>


Con el fin de detectar ambos tipos de suciedad en los panel (cafe y blanca) se realiza una serie de transformaciones que logra resaltar ambos tipos:

* Se cambia al espacio de colo CMY
* Se aplica una tranformacion gamma con parámetros a = 1.2 y Gamma = 0.4
* Luego se binariza ambas imagenes con un threshold de 240
* Se aplica una transformación morfológica de apertura para eliminar las lineas del panel que no correponden a la suciedad

Inicialmente se obtienen las listas que contienen la ubicación de las imágenes

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import *
import cv2
import sys
sys.path.append('./utils/')
from utils.GUI_Color import *
from utils.funciones_utiles import *
from pathlib import Path
import skimage.morphology as morph

# Se crea la ruta cwd para encontrar la ubicación del archivo
cwd = Path.cwd()

# Se encuentra la carpeta donde se encuentran las imágenes
DATA_PATH = cwd / 'datasets'/'Suciedad_tipo1'

# Se encuentra una lista con la ubicación de las imagenes
images = list(DATA_PATH.glob('*.jpg'))

#Se encuentra la ubicación relativa de las imágenes
images_1 =[image.relative_to(cwd) for image in images]

# Se encuentra la carpeta donde se encuentran las imágenes
DATA_PATH = cwd / 'datasets'/'Suciedad_tipo2'

# Se encuentra una lista con la ubicación de las imagenes
images = list(DATA_PATH.glob('*.jpg'))

#Se encuentra la ubicación relativa de las imágenes
images_2 =[image.relative_to(cwd) for image in images]



In [2]:
def soiling_segmentation(i):

    #####################################################
    ## Generación Máscara del panel

    # Se lee una imagen con el panel limpio
    img_rgb = cv2.cvtColor(cv2.imread(str("datasets/Limpio.jpg")),cv2.COLOR_BGR2RGB)
    # Se toma el espacio de color B del LAB
    panel_B_LAB= cv2.cvtColor(img_rgb,cv2.COLOR_RGB2LAB)[:,:,2]

    # Se genera la mascara 
    panel_mask=np.zeros_like(img_rgb)
    panel_mask[(panel_B_LAB < 116)]=True

    panel_mask = panel_mask[:,:,0]
    
    # Se aplica una erosión
    elem = morph.disk(2)
    panel_mask = morph.binary_erosion(panel_mask,elem)
    
    
    #####################################################
    ## Preprocesamiento de la imagen
    
    # Se leen las imágenes
    img_rgb_1=cv2.imread(str(images_1[i]))
    img_rgb_2=cv2.imread(str(images_2[i]))
    
    # Se cambia a un espacio de color RGB
    img_rgb_1=cv2.cvtColor(img_rgb_1, cv2.COLOR_BGR2RGB)
    img_rgb_2=cv2.cvtColor(img_rgb_2, cv2.COLOR_BGR2RGB)
    
    # Se cambia al espacio de color CMY para img_rgb_1
    img_C_CMY_1 = 255 - img_rgb_1[:,:,0]
    img_M_CMY_1 = 255 - img_rgb_1[:,:,1]
    img_Y_CMY_1 = 255 - img_rgb_1[:,:,2]

    # Se cambia al espacio de color CMY para img_rgb_2
    img_C_CMY_2 = 255 - img_rgb_2[:,:,0]
    img_M_CMY_2 = 255 - img_rgb_2[:,:,1]
    img_Y_CMY_2 = 255 - img_rgb_2[:,:,2]
    
    # Se aplica una transformación gamma 
    res_1 = gamma_correction(img_C_CMY_1, 1.2, 0.4)
    #Se aplica la expansión del histograma
    res_1 = histogram_expansion(res_1)
    
    # Se aplica una transformación gamma 
    res_2 = gamma_correction(img_C_CMY_2, 1.2, 0.4)
    #Se aplica la expansión del histograma
    res_2 = histogram_expansion(res_2)
  

    #####################################################
    ## Se aplica el threshold y se obtiene el resultado
    
    img_out_1=img_rgb_1.copy()
    img_out_2=img_rgb_2.copy()

    # Se aplica el threshold para imagen superpuesta
    img_out_1[(res_1<240)&(panel_mask==True)]=[20,200,0]
    img_out_2[(res_2<240)&(panel_mask==True)]=[20,200,0]
    
    # Se generan las mascaras del panel
    panel_mask_out1=np.zeros_like(img_out_1)
    panel_mask_out2=np.zeros_like(img_out_2)
    panel_mask_out1[(res_1<240)&(panel_mask==True)]=255 
    panel_mask_out2[(res_2<240)&(panel_mask==True)]=255 

 
    ####################################################
    ## Aplicaciones de morfología
    
    # Se aplica una apertura para eliminar segmentación
    # de las celdas del panel
    selem = morph.disk(1)
    apertura1 = morph.binary_opening(panel_mask_out1[:,:,0],selem)

    # Se aplica una apertura para eliminar segmentación
    # de las celdas del panel
    selem = morph.disk(1)
    apertura2 = morph.binary_opening(panel_mask_out2[:,:,0],selem)

    #####################################################
    ## Se presentan los resultados finales
    
    fig, arreglo_plots = plt.subplots(4,2, figsize=(15, 30))
    
    # Se presentan las imágenes originales
    arreglo_plots[0,0].imshow(img_rgb_1)
    arreglo_plots[0,1].imshow(img_rgb_2)
    
    # Se presenta el espacio de color
    arreglo_plots[1,0].imshow(res_1, cmap='gray')
    arreglo_plots[1,1].imshow(res_2, cmap='gray')
    
    # Se presenta el resultado con la imagen superpuesta
    arreglo_plots[2,0].imshow(img_out_1, cmap='gray')
    arreglo_plots[2,1].imshow(img_out_2, cmap='gray')
    
    # Se presenta la mascara en cuestion
    arreglo_plots[3,0].imshow(apertura1, cmap='gray')
    arreglo_plots[3,1].imshow(apertura2, cmap='gray')

    
    
# Se utiliza la librería de widgets de Jupyter para presentar los resultados.
_ = interact(soiling_segmentation, i=widgets.Play(value=0, min=0, max=20, step=1, interval=500))

interactive(children=(Play(value=0, description='i', interval=500, max=20), Output()), _dom_classes=('widget-i…